# Chapter 14: The Zen of Testing

## The Threat of Overfitting

In [ ]:
# The MNIST data loader that we used so far.
# It includes a training and a test set, but no validation set.

import numpy as np
import gzip
import struct


def load_images(filename):
    # Open and unzip the file of images:
    with gzip.open(filename, 'rb') as f:
        # Read the header information into a bunch of variables:
        _ignored, n_images, columns, rows = struct.unpack('>IIII', f.read(16))
        # Read all the pixels into a NumPy array:
        all_pixels = np.frombuffer(f.read(), dtype=np.uint8)
        # Reshape the pixels into a matrix where each line is an image:
        return all_pixels.reshape(n_images, columns * rows)


# 60000 images, each 785 elements (1 bias + 28 * 28 pixels)
X_train = load_images("../data/mnist/train-images-idx3-ubyte.gz")

# 10000 images, each 785 elements, with the same structure as X_train
X_test = load_images("../data/mnist/t10k-images-idx3-ubyte.gz")


def load_labels(filename):
    # Open and unzip the file of images:
    with gzip.open(filename, 'rb') as f:
        # Skip the header bytes:
        f.read(8)
        # Read all the labels into a list:
        all_labels = f.read()
        # Reshape the list of labels into a one-column matrix:
        return np.frombuffer(all_labels, dtype=np.uint8).reshape(-1, 1)


def one_hot_encode(Y):
    n_labels = Y.shape[0]
    n_classes = 10
    encoded_Y = np.zeros((n_labels, n_classes))
    for i in range(n_labels):
        label = Y[i]
        encoded_Y[i][label] = 1
    return encoded_Y


# 60K labels, each a single digit from 0 to 9
Y_train_unencoded = load_labels("../data/mnist/train-labels-idx1-ubyte.gz")

# 60K labels, each consisting of 10 one-hot encoded elements
Y_train = one_hot_encode(Y_train_unencoded)

# 10000 labels, each a single digit from 0 to 9
Y_test = load_labels("../data/mnist/t10k-labels-idx1-ubyte.gz")

In [ ]:
# The neural network, with the train() function hacked to provide
# histories of the loss and the accuracy on both the training set
# and the test set.

import numpy as np


def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def softmax(logits):
    exponentials = np.exp(logits)
    return exponentials / np.sum(exponentials, axis=1).reshape(-1, 1)


def sigmoid_gradient(sigmoid):
    return np.multiply(sigmoid, (1 - sigmoid))


def prepend_bias(X):
    return np.insert(X, 0, 1, axis=1)


def forward(X, w1, w2):
    h = sigmoid(np.matmul(prepend_bias(X), w1))
    y_hat = softmax(np.matmul(prepend_bias(h), w2))
    return (y_hat, h)


def back(X, Y, y_hat, w2, h):
    w2_gradient = np.matmul(prepend_bias(h).T, (y_hat - Y)) / X.shape[0]
    w1_gradient = np.matmul(prepend_bias(X).T,
                            np.matmul(y_hat - Y, w2[1:].T) *
                            sigmoid_gradient(h)) / X.shape[0]
    return (w1_gradient, w2_gradient)


def classify(X, w1, w2):
    y_hat, _ = forward(X, w1, w2)
    labels = np.argmax(y_hat, axis=1)
    return labels.reshape(-1, 1)


def accuracy(X, Y_unencoded, w1, w2):
    return np.average(classify(X, w1, w2) == Y_unencoded) * 100.0


def initialize_weights(n_input_variables, n_hidden_nodes, n_classes):
    np.random.seed(1234)

    w1_rows = n_input_variables + 1
    w1 = np.random.randn(w1_rows, n_hidden_nodes) * np.sqrt(1 / w1_rows)

    w2_rows = n_hidden_nodes + 1
    w2 = np.random.randn(w2_rows, n_classes) * np.sqrt(1 / w2_rows)

    return (w1, w2)


# This loss() takes different parameters than the ones in other source files
def loss(X, Y, w1, w2):
    y_hat, _ = forward(X, w1, w2)
    return -np.sum(Y * np.log(y_hat)) / Y.shape[0]


# This train() stores the loss on both the training and the test sets at each
# step. A the end, it returns those histories to the caller.
# Different from train() functions in other source files, it expects that
# _both_ Y_train and Y_test are one hot encoded.
def train(X_train, Y_train, X_test, Y_test, n_hidden_nodes, iterations, lr):
    n_input_variables = X_train.shape[1]
    n_classes = Y_train.shape[1]
    w1, w2 = initialize_weights(n_input_variables, n_hidden_nodes, n_classes)
    training_losses = []
    test_losses = []
    for i in range(iterations):
        y_hat_train, h = forward(X_train, w1, w2)
        y_hat_test, _ = forward(X_test, w1, w2)
        w1_gradient, w2_gradient = back(X_train, Y_train, y_hat_train, w2, h)
        w1 = w1 - (w1_gradient * lr)
        w2 = w2 - (w2_gradient * lr)

        training_loss = -np.sum(Y_train * np.log(y_hat_train)) / \
            Y_train.shape[0]
        training_losses.append(training_loss)
        test_loss = -np.sum(Y_test * np.log(y_hat_test)) / Y_test.shape[0]
        test_losses.append(test_loss)
        print("%5d > Training loss: %.5f - Test loss: %.5f" %
              (i, training_loss, test_loss))
    return (training_losses, test_losses, w1, w2)

In [ ]:
training_losses, test_losses, w1, w2 = train(X_train,
                                             Y_train,
                                             X_test,
                                             one_hot_encode(Y_test),
                                             n_hidden_nodes=200,
                                             iterations=10,#000,
                                             lr=0.01)

training_accuracy = accuracy(X_train, Y_train_unencoded, w1, w2)
test_accuracy = accuracy(X_test, Y_test, w1, w2)
print("Training accuracy: %.2f%%, Test accuracy: %.2f%%" %
      (training_accuracy, test_accuracy))

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

plt.plot(training_losses, label='Training set', color='blue', linestyle='-')
plt.plot(test_losses, label='Test set', color='green', linestyle='--')
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

## A Testing Conundrum

In [ ]:
# X_train/X_validation/X_test: 60K/5K/5K images
# Each image has 784 elements (28 * 28 pixels)
X_train = load_images("../data/mnist/train-images-idx3-ubyte.gz")
X_test_all = load_images("../data/mnist/t10k-images-idx3-ubyte.gz")
X_validation, X_test = np.split(X_test_all, 2)

In [ ]:
X_train.shape

In [ ]:
X_validation.shape

In [ ]:
X_test.shape

In [ ]:
# 60K labels, each a single digit from 0 to 9
Y_train_unencoded = load_labels("../data/mnist/train-labels-idx1-ubyte.gz")

# Y_train: 60K labels, each consisting of 10 one-hot encoded elements
Y_train = one_hot_encode(Y_train_unencoded)

# Y_validation/Y_test: 5K/5K labels, each a single digit from 0 to 9
Y_test_all = load_labels("../data/mnist/t10k-labels-idx1-ubyte.gz")
Y_validation, Y_test = np.split(Y_test_all, 2)

In [ ]:
Y_train.shape

In [ ]:
Y_validation.shape

In [ ]:
Y_test.shape